In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
%matplotlib notebook
%matplotlib inline
import matplotlib.dates as dates
import matplotlib.ticker as ticker
import numpy as np

df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')


# Converting data_value to tenth of celcius

df['Data_Value'] = df['Data_Value']*0.1


df['Date'] = pd.to_datetime(df['Date'])

df['Year'] = df['Date'].dt.year

df['Month_Day'] = df['Date'].dt.strftime('%m-%d')

df = df[df['Month_Day']!='02-29']
#print(df.head(20))

# Max and min for each day:
max_temp = df[(df.Year >= 2005) & (df.Year < 2015) & (df['Element'] == 'TMAX')].groupby(['Month_Day'])['Data_Value'].max()
min_temp = df[(df.Year >= 2005) & (df.Year < 2015) & (df['Element'] == 'TMIN')].groupby(['Month_Day'])['Data_Value'].min()

# Record temperature:
df = df.merge(max_temp.reset_index(drop=False).rename(columns={'Data_Value':'Max_temp'}), on='Month_Day', how='left')
df = df.merge(min_temp.reset_index(drop=False).rename(columns={'Data_Value':'Min_temp'}), on='Month_Day', how='left')

record_high = df[(df.Year==2015)&(df.Data_Value > df.Max_temp)]
record_low = df[(df.Year==2015)&(df.Data_Value < df.Min_temp)]


date_index = np.arange('2015-01-01','2016-01-01', dtype='datetime64[D]')

plt.figure()

plt.plot(date_index,max_temp,color='lightcoral', linewidth=1)
plt.plot(date_index,min_temp,color='skyblue', linewidth=1) 

plt.scatter(record_high.Date.values, record_high.Data_Value.values, color='red', s=8)
plt.scatter(record_low.Date.values, record_low.Data_Value.values, color='blue', s=8)

ax = plt.gca()
ax.axis(['2015/01/01','2015/12/31',-60,60])
plt.xlabel('Months of Year', fontsize=12)
plt.ylabel('Celsius', fontsize=12)
plt.title('Temperatures in Ann Arbour, Michigan from 2005 to 2015', fontsize=12)
plt.legend(['Record high in 2005-2014','Record low in 2005-2014','Record breaking high in 2015','Record breaking low in 2015'],loc=0,frameon=False)

ax.fill_between(date_index, max_temp, min_temp, facecolor='grey', alpha=0.40)
ax.xaxis.set_major_locator(dates.MonthLocator())
ax.xaxis.set_minor_locator(dates.MonthLocator(bymonthday=15)) 
ax.xaxis.set_major_formatter(ticker.NullFormatter())
ax.xaxis.set_minor_formatter(dates.DateFormatter('%b'))
for tick in ax.xaxis.get_minor_ticks():
    tick.tick1line.set_markersize(0) 
    tick.label1.set_horizontalalignment('center')

#plt.savefig('Temperature.pdf')
